In [15]:
# Module imports and auto-reload setup
%load_ext autoreload
%aimport if_lib, if_utils, if_dpp, if_graphics, if_consts, if_gc1dpp
%autoreload 1
import os
import json
import random

from if_utils import get_filename, show_data, save_traces

from if_lib import generate_random_challenge, read_HMAC, read_keypair, get_id_person, get_location_id, \
get_unit_id, get_resource_spec_id, get_resource, get_process, create_event, make_transfer, reduce_resource, set_user_location

from if_dpp import trace_query, check_traces, er_before, get_dpp

from if_graphics import vis_dpp, make_sankey, consol_trace

from if_gc1dpp import submit_dpp, create_sample_bike_dpp,create_sample_luffy_3d_object_dpp

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Configuration and Endpoints

In [16]:
# Define constants - must match the setup notebook
USE_CASE = 'Lauds_Machine-3D-Printer'

# Zenflows API endpoint
ENDPOINT = 'https://proxy.dpp-staging.dyne.im/zenflows/api'

# DPP service endpoint
DPP_URL = 'https://proxy.dpp-staging.dyne.im/interfacer-dpp'

# All participants
USERS = ['designer1', 'designer2', 'service_prov1', 'service_prov2', 'manufacturer1', 'manufacturer2', 'customer1', 'customer2']

## Load Setup Data from JSON Files

In [17]:
# Calculate file paths
USERS_FILE = get_filename('cred_users.json', ENDPOINT, USE_CASE)
LOCS_FILE = get_filename('loc_users.json', ENDPOINT, USE_CASE)
UNITS_FILE = get_filename('units_data.json', ENDPOINT, USE_CASE)
SPECS_FILE = get_filename('res_spec_data.json', ENDPOINT, USE_CASE)
DPP_FILE = get_filename('dpp_data.json', ENDPOINT, USE_CASE)
RES_FILE = get_filename('initial_resources.json', ENDPOINT, USE_CASE)
PROCESS_FILE = get_filename('process_data.json', ENDPOINT, USE_CASE)

# Load all data
print("Loading setup data...")

with open(USERS_FILE, 'r') as f:
    users_data = json.loads(f.read())
print(f"✓ Loaded {len(users_data)} users")

with open(LOCS_FILE, 'r') as f:
    locs_data = json.loads(f.read())
print(f"✓ Loaded {len(locs_data)} locations")

with open(UNITS_FILE, 'r') as f:
    units_data = json.loads(f.read())
print(f"✓ Loaded {len(units_data)} units")

with open(SPECS_FILE, 'r') as f:
    res_spec_data = json.loads(f.read())
print(f"✓ Loaded {len(res_spec_data)} resource specifications")

with open(PROCESS_FILE, 'r') as f:
    process_data = json.loads(f.read())
print(f"✓ Loaded {len(process_data)} processes")

with open(RES_FILE, 'r') as f:
    res_data = json.loads(f.read())
print(f"✓ Loaded {len(res_data)} initial resources")

# Initialize event sequence and DPP data
event_seq = []
dpp_data = {}

print("\n✓ All setup data loaded successfully!")

Loading setup data...
✓ Loaded 2 users
✓ Loaded 2 locations
✓ Loaded 5 units
✓ Loaded 5 resource specifications
✓ Loaded 1 processes
✓ Loaded 5 initial resources

✓ All setup data loaded successfully!


## Verify Loaded Components

Let's verify that all the bike components were successfully loaded from the setup notebook.

In [18]:
# Verify that all components are available
print("Checking for required components...")
required_components = ['3D-object-Luffy']

all_present = True
for component in required_components:
    if component in res_data:
        print(f"✓ {component}: {res_data[component]['id']}")
    else:
        print(f"✗ {component}: NOT FOUND")
        all_present = False

if all_present:
    print(f"\n✓ All components loaded successfully! Ready to create the bike.")
else:
    print(f"\n✗ Some components are missing. Please run the setup notebook first.")

Checking for required components...
✓ 3D-object-Luffy: 06E0HENR5WEJBR71GZGW5V9G04

✓ All components loaded successfully! Ready to create the bike.


## Create GC1DPP Digital Product Passport (BEFORE Production)

**This is the key innovation**: Create and submit the Digital Product Passport **BEFORE** producing the final bike.

In [19]:
# Create the GC1DPP data for the fancy 3D-object-Luffy (BEFORE producing it)
bike_dpp = create_sample_luffy_3d_object_dpp()

# Customize the DPP data with information about the components we've already created
bike_dpp['productOverview']['productName']['value'] = 'fancy 3D-object-Luffy'
bike_dpp['productOverview']['productDescription']['value'] = f"A unique 3D-object-Luffy"

# Add component information based on the resources we created
bike_dpp['components'] = [
    {
        "componentDescription": {"type": "string", "value": res_data['3D-object-Luffy']['name']},
        "componentGTIN": {"type": "string", "value": res_data['3D-object-Luffy']['res_ref_id']}
    },

]

# Update economic operator with actual user info
bike_dpp['economicOperator']['companyName']['value'] = users_data['A']['name']
bike_dpp['economicOperator']['addressLine1']['value'] = locs_data['A']['addr'].split(',')[0]
bike_dpp['economicOperator']['addressLine2']['value'] = ','.join(locs_data['A']['addr'].split(',')[1:])

print("✓ DPP data created (3D-object-Luffy)")
print(json.dumps(bike_dpp, indent=2))

✓ DPP data created (3D-object-Luffy)
{
  "productOverview": {
    "brandName": {
      "type": "string",
      "value": "Straw Hat Studios"
    },
    "productName": {
      "type": "string",
      "value": "fancy 3D-object-Luffy"
    },
    "productDescription": {
      "type": "string",
      "value": "A unique 3D-object-Luffy"
    },
    "countryOfOrigin": {
      "type": "string",
      "value": "Japan"
    },
    "color": {
      "type": "string",
      "value": "Multi-color"
    },
    "netWeight": {
      "type": "number",
      "value": 1.2,
      "units": "kg"
    },
    "modelName": {
      "type": "string",
      "value": "LUFFY-3D-2025"
    }
  },
  "reparability": {
    "availabilityOfSpareParts": {
      "type": "string",
      "value": "Limited parts available through licensed distributor"
    }
  },
  "environmentalImpact": {
    "co2eEmissionsPerUnit": {
      "type": "number",
      "value": 8,
      "units": "kg"
    },
    "minimumContentOfMaterialWithSustainability

## Submit DPP to GC1DPP Service

In [20]:
# Submit the DPP to the DPP service BEFORE creating the bike
try:
    dpp_ulid = submit_dpp(
        bike_dpp,
        users_data['A']['eddsa_public_key'],
        users_data['A']['keyring']['eddsa'],
        DPP_URL
    )
    
    print(f"✓ DPP submitted successfully with ULID: {dpp_ulid}")
    
    # Store the DPP ULID - we'll use it when creating the bike
    bike_metadata = {
        'dpp': dpp_ulid
    }
    
    print(f"✓ DPP ULID ready to be added to bike metadata: {dpp_ulid}")
    
except Exception as e:
    print(f"✗ Error submitting DPP: {e}")
    print("Note: Make sure the DPP service is running at", DPP_URL)
    dpp_ulid = None
    bike_metadata = {}

Submitting DPP to https://proxy.dpp-staging.dyne.im/interfacer-dpp/dpp
Public key: b3XYnrMAPYre61K2jH6g...
Signature: HbhPZd0lmA71A1XloM/eULxllNnKWR+yNJYCMy7T...
DPP submitted with ULID: 01KG4BP81JNDK92AKTD909JWBP
✓ DPP submitted successfully with ULID: 01KG4BP81JNDK92AKTD909JWBP
✓ DPP ULID ready to be added to bike metadata: 01KG4BP81JNDK92AKTD909JWBP


## Produce the 3D-object-Luffy (with DPP Metadata)

Now we produce the final 3D-object-Luffy, embedding the DPP ULID in its metadata at creation time.

In [21]:
# Debug: Check what we're about to send
print("3D-object-Luffy_metadata content:")
print(bike_metadata)
print("\nType:", type(bike_metadata))
if bike_metadata:
    print("dppUlid:", bike_metadata.get('dpp'))

3D-object-Luffy_metadata content:
{'dpp': '01KG4BP81JNDK92AKTD909JWBP'}

Type: <class 'dict'>
dppUlid: 01KG4BP81JNDK92AKTD909JWBP


In [ ]:
# # Produce the fancy collaborative bike with all consumption/cite events
# cur_res = action = event_note = amount = cur_pros = None
# cur_pros = process_data['Create_3D_object_Luffy']

# action = 'consume'
# event_note = 'consume filament for 3D-object-Luffy'
# amount = 50  # example: 50 grams, adjust as needed
# cur_pros = process_data['Create_3D_object_Luffy']

# cur_res = res_data['Filament']  # pre-existing

# event_id, ts = create_event(
#     provider=users_data['A'],
#     action='consume',                   # action on existing resource
#     note='consume filament for 3D-object-Luffy',
#     amount=50,
#     process=process_data['Create_3D_object_Luffy'],
#     res_spec_data=res_spec_data,
#     existing_res=cur_res,              # use existing_res, NOT new_res
#     endpoint=ENDPOINT
# )
# event_seq.append({'ts': ts, 'event_id': event_id, 'action': action, 'res_name': cur_res['name'], 'res': cur_res['id']})

# # Define event consume for electrical energy
# cur_res = action = event_note = amount = cur_pros = None
# action = 'consume'
# event_note = 'consume electrical energy for 3D-object-Luffy'
# amount = 500  # example: 500 Wh, adjust as needed
# cur_pros = process_data['Create_3D_object_Luffy']

# cur_res = res_data['Electrical-Energy']

# event_id, ts = create_event(users_data['A'], action, event_note, amount=amount, process=cur_pros,
#                  res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

# event_seq.append({'ts': ts, 'event_id': event_id, 'action': action, 'res_name': cur_res['name'], 'res': cur_res['id']})

# # con# Define event cite for 3D gcode file
# cur_res = action = event_note = amount = cur_pros = None
# action = 'cite'
# event_note = 'cite 3D-gcode-file-Luffy'
# amount = 1
# cur_pros = process_data['Create_3D_object_Luffy']

# cur_res = res_data['3D-gcode-file-Luffy']

# event_id, ts = create_event(users_data['A'], action, event_note, amount=amount, process=cur_pros,
#                  res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

# event_seq.append({'ts': ts, 'event_id': event_id, 'action': action, 'res_name': cur_res['name'], 'res': cur_res['id']})

# # Define event produce for 3D object “Luffy”
# cur_res = action = event_note = amount = cur_pros = None
# action = 'produce'
# event_note = 'produce 3D-object-Luffy'
# amount = 1
# cur_pros = process_data['Create_3D_object_Luffy']

# res_data['3D-object-Luffy'] = {
#     "res_ref_id": f'3D-object-Luffy-{random.randint(0, 10000)}',
#     "name": '3D-object-Luffy',
#     "spec_id": res_spec_data['3D-object-Luffy']['id']
# }

# print(res_data['3D-object-Luffy'])
# cur_res = res_data['3D-object-Luffy']

# event_id, ts = create_event(users_data['A'], action, event_note, amount=amount, process=cur_pros,
#                  res_spec_data=res_spec_data, new_res=cur_res, endpoint=ENDPOINT)

# event_seq.append({'ts': ts, 'event_id': event_id, 'action': action, 'res_name': cur_res['name'], 'res': cur_res['id']})
# event_seq.append({'ts': ts, 'process_id':cur_pros['id'], 'name' : cur_pros['name']})
# print(f"\n✓ Bike created with ID: {cur_res['id']}")
# if bike_metadata:
#     print(f"✓ Bike metadata includes DPP ULID: {bike_metadata.get('dpp', 'N/A')}")

{'res_ref_id': '3D-object-Luffy-8188', 'name': '3D-object-Luffy', 'spec_id': '06DZMBJ9C45H8HMYYPBJ8KY880'}

✓ Bike created with ID: 06DZMBRKBHMMKRRNRGH3FBZVF8
✓ Bike metadata includes DPP ULID: None


## Save DPP Data for Future Reference

In [22]:
# Save the DPP data to file for future reference
if dpp_ulid:
    res_data['3D-object-Luffy']['dpp_ulid'] = dpp_ulid
    dpp_data['3D-object-Luffy'] = {
        'ulid': dpp_ulid,
        'resource_id': res_data['3D-object-Luffy']['id'],
        'resource_ref_id': res_data['3D-object-Luffy']['res_ref_id'],
        'dpp': bike_dpp
    }
    
    # Save DPP data to file
    if os.path.isfile(DPP_FILE):
        with open(DPP_FILE, 'r') as f:
            existing_dpp_data = json.loads(f.read())
    else:
        existing_dpp_data = {}
    
    existing_dpp_data.update(dpp_data)
    
    with open(DPP_FILE, 'w') as f:
        json.dump(existing_dpp_data, f, indent=2)
    
    print(f"✓ DPP data saved to {DPP_FILE}")
else:
    print("⚠ No DPP ULID available to save")

✓ DPP data saved to use_cases/Lauds_Machine-3D-Printer/proxy.dpp-staging.dyne.im%2Fzenflows%2Fapi/dpp_data.json


## Display Summary Data

In [23]:
# Display summary of created resources and DPP
show_data(users_data, locs_data, res_data, units_data, res_spec_data, process_data, event_seq)

Users
{
  "A": {
    "userChallenges": {
      "whereParentsMet": "London",
      "nameFirstPet": "Fuffy",
      "nameFirstTeacher": "Jim",
      "whereHomeTown": "Paris",
      "nameMotherMaid": "Wright"
    },
    "name": "User A",
    "username": "userA_username",
    "email": "userA@example.org",
    "note": "me.userA.org",
    "seedServerSideShard.HMAC": "IqaW9oqMFSNgrjXsU2U/EwT4FugadCm8ZVFqkcT4spc=",
    "seed": "solution garage know special trap wheel timber raven measure miracle achieve horn",
    "eddsa_public_key": "b3XYnrMAPYre61K2jH6gq3A4G6xZrKYfuZ9kwihs55S",
    "keyring": {
      "eddsa": "DCqKRyK14C3Jmk4ifpGgMQFcv8nznouynSYXA8RMRieC"
    },
    "id": "06DY599GHZ0Y3XH1NTAW2TY4GM",
    "location_id": "06E0HEM4KFN6RQVN7V2BV2AT40"
  },
  "B": {
    "userChallenges": {
      "whereParentsMet": "Amsterdam",
      "nameFirstPet": "Toby",
      "nameFirstTeacher": "Juliet",
      "whereHomeTown": "Rome",
      "nameMotherMaid": "Banks"
    },
    "name": "User B",
    "username"

## Supply Chain Tracing and Visualization

### Backward Tracing: Find All Inputs

In [24]:
trace_me = res_data['3D-object-Luffy']['id']
print(f"Resource to be traced: {trace_me}")
tot_dpp = []
visited = set()
er_before(trace_me, users_data['A'], dpp_children=tot_dpp, depth=0, visited=visited, endpoint=ENDPOINT)

# Serializing json
json_object = json.dumps(tot_dpp, indent=2)

print(json_object)
print(f"\n✓ Traced {len(visited)} resources")

Resource to be traced: 06E0HENR5WEJBR71GZGW5V9G04
[
  {
    "accountingQuantity": {
      "hasNumericalValue": "1",
      "hasUnit": {
        "id": "06E0HEMDRGE0944KEG0W0JNSH8",
        "label": "u_piece",
        "symbol": "om2:one"
      }
    },
    "currentLocation": {
      "alt": "0",
      "id": "06E0HEM4KFN6RQVN7V2BV2AT40",
      "lat": "52.35871773455108",
      "long": "4.916762398221842",
      "mappableAddress": "Oosterpark 9, 1091 AC Amsterdam",
      "name": "OLVG",
      "note": "location.user1.org"
    },
    "custodian": {
      "id": "06DY599GHZ0Y3XH1NTAW2TY4GM",
      "name": "User A",
      "note": null,
      "primaryLocation": {
        "alt": "0",
        "id": "06E0HEM4KFN6RQVN7V2BV2AT40",
        "lat": "52.35871773455108",
        "long": "4.916762398221842",
        "mappableAddress": "Oosterpark 9, 1091 AC Amsterdam",
        "name": "OLVG",
        "note": "location.user1.org"
      },
      "type": "Person"
    },
    "id": "06E0HENR5WEJBR71GZGW5V9G04",
 

### Query DPP from Zenflows

In [25]:
be_dpp = get_dpp(trace_me, endpoint=ENDPOINT)
print(json.dumps(be_dpp, indent=2))

[
  {
    "node": {
      "accountingQuantity": {
        "hasNumericalValue": "1",
        "hasUnit": {
          "id": "06E0HEMDRGE0944KEG0W0JNSH8"
        }
      },
      "classifiedAs": null,
      "conformsTo": {
        "id": "06E0HEN30JT66WW95TAZZDYNG0"
      },
      "containedIn": {
        "id": null
      },
      "currentLocation": {
        "id": "06E0HEM4KFN6RQVN7V2BV2AT40"
      },
      "custodian": {
        "id": "06DY599GHZ0Y3XH1NTAW2TY4GM"
      },
      "id": "06E0HENR5WEJBR71GZGW5V9G04",
      "license": null,
      "licensor": null,
      "lot": {
        "id": null
      },
      "metadata": null,
      "name": "3D-object-Luffy",
      "note": null,
      "okhv": null,
      "onhandQuantityHas": {
        "hasUnit": {
          "id": "06E0HEMDRGE0944KEG0W0JNSH8"
        },
        "numericalValue": "1"
      },
      "previousEvent": {
        "id": "06E0HENR5F94R1GQJRGXB8X2ZC"
      },
      "primaryAccountable": {
        "id": "06DY599GHZ0Y3XH1NTAW2TY4GM"
  

### Trace Validation

In [27]:
trace = trace_query(trace_me, endpoint=ENDPOINT)
check_traces(trace, event_seq, tot_dpp, be_dpp)

################################################################################
nr trace: 12, nr events: 0, nr front-end dpp: 12, nr back-end dpp: 12
################################################################################
Check whether there are any duplicated trace items
################################################################################
Check whether there are any duplicated events
################################################################################
Check whether there are any duplicated nodes in front-end dpp
################################################################################
Check whether there are any duplicated nodes in back-end dpp
################################################################################
Are trace items in the events?
NOT FOUND: trace item 3D-object-Luffy id: 06E0HENR5WEJBR71GZGW5V9G04 of type EconomicResource
NOT FOUND: trace item produce id: 06E0HENR5F94R1GQJRGXB8X2ZC of type EconomicEvent
NOT FOUND: trace

### Save Trace Data

In [28]:
save_traces(USE_CASE, tot_dpp, trace, be_dpp, event_seq)
print("✓ Trace data saved")

✓ Trace data saved


### Visualize Supply Chain with Sankey Diagram

In [30]:
labels = []
sources = []
targets = []
values = []
color_nodes = []
color_links = []
assigned = {}
vis_dpp(tot_dpp[0], count=0, assigned=assigned, labels=labels, targets=targets, sources=sources, values=values, color_nodes=color_nodes, color_links=color_links)
sources, targets = consol_trace(assigned, sources, targets)
make_sankey(sources, targets, labels, values, color_nodes, color_links)

## Production Complete

The collaborative bike has been successfully produced with a Digital Product Passport!

### What we accomplished:

1. ✓ Loaded pre-produced components from setup notebook
2. ✓ Created a GC1DPP Digital Product Passport
3. ✓ Submitted the DPP to the service and received a ULID
4. ✓ Produced the bike with the DPP ULID embedded in metadata
5. ✓ Traced the complete supply chain
6. ✓ Visualized the material flows

### Key Integration Point:

The bike's metadata in Zenflows contains:
```json
{
  "dpp": "01KAX45XX1XSACK6PG3W5FW04Z"
}
```

This ULID links the ValueFlows economic events with the GC1DPP passport, enabling:
- Complete supply chain traceability
- Regulatory compliance
- Circular economy tracking
- Cryptographic verification